# 

# Deep Convolutional Text-To-Speech

In [ ]:
!ls

sample_data


### Cloning Code

In [ ]:
!git clone https://github.com/Kyubyong/dc_tts.git

Cloning into 'dc_tts'...
remote: Enumerating objects: 168, done.
remote: Total 168 (delta 0), reused 0 (delta 0), pack-reused 168
Receiving objects: 100% (168/168), 3.13 MiB | 8.31 MiB/s, done.
Resolving deltas: 100% (95/95), done.


In [ ]:
!cd dc_tts

dc_tts	sample_data


## Install Libraries

In [ ]:
!pip install numpy
!pip install tensorflow
!pip install librosa
!pip install tqdm
!pip install matplotlib
!pip install scipy

## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install Libraries

In [ ]:
%cd /content/drive/MyDrive/
!wget https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
!tar -xf LJSpeech-1.1.tar.bz2


!wget http://www.caito.de/data/Training/stt_tts/ru_RU.tgz
!tar -zxvf ru_RU.tgz
!mv ru_RU M_AILABS_Ru

In [ ]:
%cd /content/dc_tts
!ls

/content/dc_tts
data_load.py	       hyperparams.py  networks.py  synthesize.py
fig		       LICENSE	       prepo.py     train.py
harvard_sentences.txt  modules.py      README.md    utils.py


## Data Preprocessing

In [ ]:
import random
"""
Move all wav file to the common folder
Concat all text files into one
"""
def refactor_M_AILABS(sex='female', speaker_index=0
                      , dataset_dir="/content/drive/MyDrive/M_AILABS_Ru/"
                      , out_dir="/content/drive/MyDrive/M_AILABS_Ru/"
                      , move_files=True
                      ):
  print(" *** Start refactoring... ***")
  path = f"{dataset_dit}{''if dataset_dit[-1] == '/' else '/'}by_book/{sex}"
  %cd $path
  speakers = !ls
  books_path = f"{path}/{speakers[speaker_index]}"
  %cd $books_path
  books_folders = !ls -d */ | cut -f1 -d'/'
  path = f"{out_dir}{''if out_dir[-1] == '/' else '/'}"
  %cd $path
  !mkdir wavs
  wavs_path = f"{path}wavs/"
  !touch mlbs_audio_text.txt
  for book in books_folders:
    cur_wavs_path = f"{books_path}/{book}/wavs/*"
    cur_text_path = f"{books_path}/{book}/metadata.csv"
    !cat $cur_text_path >> mlbs_audio_text.txt
    if move_files:
      !mv -v $cur_wavs_path $wavs_path
  print(" *** Refactoring finished! ***")

In [ ]:
"""
M-Ailabs dataset has 2 strings of audio trascription -
raw and normalized
This function chooose one, based on argument 
and add wav file full path
"""
def transform_m_ailabs_text(file_name="mlbs_audio_text.txt",
                            new_file_name="mlbs_audio_text_transformed.txt",
                            dataset_dir="/content/drive/MyDrive/M_AILABS_Ru/",
                            raw_data=False):
  write_file = open(dataset_dir+new_file_name, 'w', encoding='utf-8')
  with open(dataset_dir+file_name, 'r', encoding='utf-8') as read_file:
    for line in read_file:
      line = line.split('|')
      wav_file_path = f"{dataset_dir}wavs/{line[0]}.wav"
      new_line = f"{wav_file_path}|{line[1] if raw_data else line[2]}"
      write_file.write(new_line)

## GPU INFO

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, 
and then re-execute this cell.


## Audio Preprocessing

In [ ]:
!python prepo.py

2021-05-23 14:09:40.854198: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
100% 13100/13100 [16:07<00:00, 13.54it/s]


## Reinstall TensorFlow

In [ ]:
!pip install --force-reinstall tensorflow

     |████████████████████████████████| 454.3MB 26kB/s 
     |████████████████████████████████| 4.0MB 15.7MB/s 
     |████████████████████████████████| 61kB 7.4MB/s 
     |████████████████████████████████| 4.0MB 35.4MB/s 
     |████████████████████████████████| 133kB 42.0MB/s 
     |████████████████████████████████| 1.2MB 33.4MB/s 
     |████████████████████████████████| 1.0MB 32.0MB/s 
     |████████████████████████████████| 6.0MB 15.9MB/s 
     |████████████████████████████████| 14.8MB 298kB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 471kB 39.3MB/s 
     |████████████████████████████████| 153kB 45.2MB/s 
     |████████████████████████████████| 4.9MB 29.6MB/s 
     |████████████████████████████████| 296kB 41.9MB/s 
     |████████████████████████████████| 61kB 7.7MB/s 
     |████████████████████████████████| 829kB 38.1MB/s 
     |████████████████████████████████| 102kB 10.3MB/s 


## Train TextToMel

In [ ]:
!python train.py 1


2021-05-23 14:43:50.116032: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-05-23 14:43:53.060407: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-05-23 14:43:53.073215: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-05-23 14:43:53.073281: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a78b674cb53e): /proc/driver/nvidia/version does not exist
2021-05-23 14:43:53.073806: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Instru

## Train SSRN


In [ ]:
!python train.py 2

## Synthesize audio


In [ ]:
!python synthesize.py